### 数据库

In [1]:
import sys
import re
from pyhive import hive
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import scipy.stats as stats
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
import requests
import json
import time
from matplotlib import pyplot as plt

from matplotlib.font_manager import FontProperties
font = FontProperties('simhei', size=20) #中文字体


pd.set_option('display.max_columns', None) #显示所有列
sys.path.append('/data/datamining/Py_functions/get_data')
from get_data_func import * ##取数function  

# 取数

In [2]:
pcr_bi_rawdata = pd.read_excel('PCR蓝耳2021-2022.xlsx')
pcr_bi_rawdata = pcr_bi_rawdata.sort_values(by = '采样时间')

# 训练数据
繁殖场，工段为母猪和仔猪，一个月内（当前时间前30天到现在的表现）

In [3]:
# 基础表。按照要求先筛选符合的场区信息。
field_base_sql = '''SELECT ffieldid, ffield, fareaid, farea FROM myana.datamining_DIM_AF WHERE ffieldtypename = '繁殖场' '''
hiveConn = hive.Connection(host='10.106.20.15', port=10000, username='szchenye', password='szcy230#',
                                               database='mydw', auth='CUSTOM')

field_base_df = pd.read_sql(field_base_sql,hiveConn)
field_base_df

,ffieldid,ffield,fareaid,farea
0,15539915,广垦牧原2场繁殖场,15539913,None
1,15524202,清江浦牧原1场繁殖场,15429781,清江浦牧原
2,15470593,赣榆牧原2场繁殖场,15445911,赣榆牧原
3,15338639,扎旗牧原2场繁殖场,15338635,扎旗牧原
4,15439304,前郭4场繁殖场3区,15332317,前郭牧原
...,...,...,...,...
708,15425921,内乡28场繁殖场,13903,内乡牧原
709,15521126,康平11场繁殖场,15380707,康平牧原
710,15608210,公安牧原9场繁殖场,15365951,公安牧原
711,15337544,界首牧原6场繁殖场,15337171,界首牧原


该场区母猪来源涉及场区个数：一个月内该场区的母猪来源涉及到多少个不同的场区。“该场区”指该批次所属场区情况。一个月指当前时间前1个月到现在的时间内:2021-12-15到2022-01-15， 母猪批次包括：后备，怀孕，待配，泌乳，空怀

In [4]:
currenttime=time.strftime('%Y-%m-%d',time.localtime(time.time())) #当前时间-年月日
#该场区母猪来源涉及场区个数：一个月内该场区的母猪来源涉及到多少个不同的场区 
#该字段的意思是：这个月转群进这个场区的母猪批次，每个批次的来源场区个数。理想状态下是：一个繁殖场的母猪来源只来源于一个场区，并且这个场区是配套的生长场
# 用到一个月内的转群表

mom_source_fieldnum_sql = '''SELECT t.to_fbatchno,  t.to_ffieldid, t.to_segmentid, count(DISTINCT t.ffieldid) AS source_fieldnum FROM (
SELECT fbatchno, to_fbatchno, ffieldid, to_ffieldid, to_segmentid FROM myana.ybb_batch_retrospect_change_08
WHERE to_segmentid IN (13910, 13909, 21254,29535, 13908 )
AND last_changedate BETWEEN '2021-12-15' AND '2022-01-15') t 
GROUP BY t.to_fbatchno, t.to_ffieldid , t.to_segmentid '''

mom_source_fieldnum_df = pd.read_sql(mom_source_fieldnum_sql,hiveConn)
mom_source_fieldnum_df 

,to_fbatchno,to_ffieldid,to_segmentid,source_fieldnum
0,AH0430C00106217223220102005,15334489,13910,1
1,CH00005104275211221007,15482930,13909,1
2,CX1721010006087369220101022,15116201,13910,1
3,DL0523C00205154914211231077,15396973,13909,1
4,DL3704222715211228016,15176675,13908,1
...,...,...,...,...
19043,YZNX00006171920211225012,15483862,13910,1
19044,ZC00006091610211227015,15457228,13910,1
19045,ZP1324C00208064537220104030,15439668,21254,1
19046,ZX1205202411211101031,15171360,13909,1


字段2：该场区后备猪批次进群前是否检出蓝耳抗原。一个月内场区后备批次有检出蓝耳的批次个数。时间为2021-12-15到2022-01-15laner_houbei_batch， 送检目的是转群监控

In [5]:
#筛选出后备批次在这一个月内检出蓝耳的场区
laner_houbei_batch = pcr_bi_rawdata.loc[(pcr_bi_rawdata.采样时间 >= '2021-12-15')&(pcr_bi_rawdata.采样时间 <= '2022-01-15')&(pcr_bi_rawdata.工段 == '后备')&(pcr_bi_rawdata.送检目的 == '转群监控')]
laner_houbei_batch['laner_houbei'] = 1
laner_houbei_batch = laner_houbei_batch.drop_duplicates(subset = ['批次编号'])

/tmp/ipykernel_15563/2701979129.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laner_houbei_batch['laner_houbei'] = 1


字段3：一胎怀孕母猪批次（后备）配种的场区类型， 目的：看该怀孕批次母猪配种地方是生长场/繁殖场。
一胎怀孕母猪 = 后备 不改
一胎怀孕母猪 = 怀孕了好几次的母猪，泌乳母猪，空怀，待配， 要追溯到第一次她怀孕的时候的场区

1. 先判断该批次是否是一胎怀孕母猪 = 后备, 在群状态 = 1
2. 该批次配种的环境。大部分后备批次都会转群到待配批次29535，那是后备配种的工段，其所属的场区是配种场区。记录场区类型


In [6]:
houbei_field_sql = '''
SELECT A.*, B.ffieldtypename FROM (
SELECT a.fbatchno, a.ffieldid, a.to_ffieldid FROM myana.ybb_batch_retrospect_change_08 a 
inner JOIN 
(SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13908) b 
ON   a.fbatchno = b.fbatchno AND a.ffieldid = b.ffieldid 
WHERE a.to_segmentid = 29535 ) A 
LEFT JOIN  myana.datamining_DIM_AF B  ON A.to_ffieldid = B.ffieldid '''

houbei_field_df = pd.read_sql(houbei_field_sql,hiveConn)
houbei_field_df


,fbatchno,ffieldid,to_ffieldid,ffieldtypename
0,LSX00004193013211222003,15426357,15426357,繁殖场
1,LZ0000420487X211204002,15417702,15417702,繁殖场
2,NJ00004066370211201023,15398800,15398800,繁殖场
3,NJ00004066370211201023,15398800,15398800,繁殖场
4,NXZHT00004034317211123003,15418263,15418263,综合场
...,...,...,...,...
2910,SY0924C00304087911220105006,15472553,15472553,繁殖场
2911,ZY00004062510211220001,15470030,15470030,繁殖场
2912,GZ6804162032211215002,15277996,15277996,繁殖场
2913,SL0382C00204100631220114002,15335524,15335524,繁殖场


In [7]:
#大圈的怀孕批次
dajuan_batch = houbei_field_df.loc[~(houbei_field_df.ffieldtypename == '繁殖场')]
dajuan_batch['大圈配种怀孕批次']='是'
dajuan_batch = dajuan_batch.loc[:, ['fbatchno', 'ffieldid', '大圈配种怀孕批次']].drop_duplicates()
dajuan_batch

/tmp/ipykernel_15563/2673267294.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dajuan_batch['大圈配种怀孕批次']='是'


,fbatchno,ffieldid,大圈配种怀孕批次
4,NXZHT00004034317211123003,15418263,是
21,KD00004173217211214031,15431119,是
24,GN0724O00204054412220106008,15473570,是
52,ZX0881130004115955220106003,15277769,是
55,JC1602M00104127174220111019,15390771,是
...,...,...,...
2808,GY00004293133210928017,15470597,是
2821,FX0000420821X211227010,15468088,是
2840,NXZHT00004033113211118018,15418271,是
2843,NXZHT1391M00204012019220107003,15418257,是


In [8]:
# 待配和怀孕批次
daipei_batch_sql = ''' SELECT A.*, B.ffieldtypename FROM (
SELECT a.fbatchno, b.fbatchno AS source_fbatchno , b.fsegmentid, b.to_segmentid ,b.to_ffieldid AS 待配场区ID FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid in (13909,29535) ) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno WHERE b.fsegmentid = 13908) A 
LEFT JOIN myana.datamining_DIM_AF B  ON A.待配场区ID = B.ffieldid '''
daipei_batch_df = pd.read_sql(daipei_batch_sql,hiveConn)

#大圈的怀孕批次
dajuan_batch2 = daipei_batch_df.loc[~(daipei_batch_df.ffieldtypename == '繁殖场')]
dajuan_batch2['大圈配种怀孕批次']='是'
dajuan_batch2 = dajuan_batch2.loc[:, ['fbatchno', '待配场区id', '大圈配种怀孕批次']].drop_duplicates()
dajuan_batch2


/tmp/ipykernel_15563/4057681678.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dajuan_batch2['大圈配种怀孕批次']='是'


,fbatchno,待配场区id,大圈配种怀孕批次
247,FY00009070814211117007,15432241,是
359,TH740901183473200101,15168144,是
465,WK140917221263200509,15340831,是
505,BY0000936246961210728,15432013,是
571,WK3809083619211116001,15358601,是
...,...,...,...
14908,HA0621M00109221631220101001,15526732,是
14919,GN00009192374211013009,15438879,是
14979,NXZHT0000925322755210821,15418281,是
15193,KD0000936210691210711,15380896,是


In [9]:
# 其他母猪批次，需要追述到第一次怀孕的地点（也就是13908后备 转到29535待配时，待配的场区类型）
other_mom_batch_sql = '''SELECT abc.*, f.ffieldtypename FROM (
SELECT DISTINCT t.fbatchno, t.source2_fieldid AS 待配场区id FROM (
SELECT aaa.*, bbb.fbatchno AS source3_fbatchno,bbb.fsegmentid AS source3_segmentid, bbb.ffieldid AS source3_fieldid FROM(
SELECT aa.*, bb.fbatchno AS source_source_fbatchno, bb.fsegmentid AS source2_segmentid, bb.ffieldid AS source2_fieldid FROM (
SELECT a.fbatchno, b.fbatchno AS source_fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid, b.ffieldid AS source_fieldid FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid in (13910,21254)) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno ) aa 
LEFT JOIN  myana.ybb_batch_retrospect_change_08 bb ON aa.source_fbatchno = bb.to_fbatchno ) aaa 
LEFT JOIN myana.ybb_batch_retrospect_change_08 bbb ON aaa.source_source_fbatchno = bbb.to_fbatchno ) t 
WHERE t.source2_segmentid = 29535 AND t.source3_segmentid = 13908) abc 
LEFT JOIN  myana.datamining_DIM_AF f ON abc.待配场区id = f.ffieldid'''

other_mom_batch = pd.read_sql(other_mom_batch_sql,hiveConn)
other_mom_batch

,fbatchno,待配场区id,ffieldtypename
0,XB1302C0010631301X220110003,15440816,生长场
1,AX0000801196487210803,15457849,繁殖场
2,XP1721C00106187860220102017,15338583,繁殖场
3,XXT00006102643211208007,15472579,繁殖场
4,WM0000811194387210530,15440818,繁殖场
...,...,...,...
8693,ZY0683C00306250607220121001,15374551,生长场
8694,TY0822C00106120431220105011,15330383,繁殖场
8695,DA0000805185802200715,15385869,None
8696,ZQ0526C00206125525220114017,15338639,繁殖场


In [10]:
dajuan_batch3 = other_mom_batch.loc[~(other_mom_batch.ffieldtypename == '繁殖场')]
dajuan_batch3['大圈配种怀孕批次']='是'
dajuan_batch3 = dajuan_batch3.loc[:, ['fbatchno', '大圈配种怀孕批次']].drop_duplicates()
dajuan_batch3

/tmp/ipykernel_15563/2504526901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dajuan_batch3['大圈配种怀孕批次']='是'


,fbatchno,大圈配种怀孕批次
0,XB1302C0010631301X220110003,是
5,JP132200320606122X220117009,是
12,NXZHT1391M01606312413220108008,是
19,YQ630802214682200704,是
23,DZ1381130006260653220101007,是
...,...,...
8679,LZ0882C01706195731220112012,是
8687,KL7606101426211224027,是
8689,NXZHT1391M01806015731220113009,是
8690,KP0123C00206197332220117008,是


In [11]:
dajuan_batch_total = dajuan_batch.append(dajuan_batch3, ignore_index = True)
dajuan_batch_total

,fbatchno,ffieldid,大圈配种怀孕批次
0,NXZHT00004034317211123003,15418263.0,是
1,KD00004173217211214031,15431119.0,是
2,GN0724O00204054412220106008,15473570.0,是
3,ZX0881130004115955220106003,15277769.0,是
4,JC1602M00104127174220111019,15390771.0,是
...,...,...,...
1813,LZ0882C01706195731220112012,NaN,是
1814,KL7606101426211224027,NaN,是
1815,NXZHT1391M01806015731220113009,NaN,是
1816,KP0123C00206197332220117008,NaN,是


字段4：该场区怀孕批次检出蓝耳抗原个数。一月内该场区怀孕批次检出蓝耳抗原个数

In [12]:
laner_huaiyun_batch = pcr_bi_rawdata.loc[(pcr_bi_rawdata.采样时间 >= '2021-12-15')&(pcr_bi_rawdata.采样时间 <= '2022-01-15')&(pcr_bi_rawdata.批次编号.notnull())&(pcr_bi_rawdata.工段 == '怀孕')&(pcr_bi_rawdata.批次编号 != '/')]
laner_huaiyun_batch['laner_huaiyun'] = 1

 #蓝耳抗原数量
laner_num_array = laner_huaiyun_batch.loc[:, ['批次编号', 'laner_huaiyun']].groupby(['批次编号']).sum().laner_huaiyun.values
print(laner_num_array) #展示

# 创建新的dataframe 
base_df_feature4 = pd.DataFrame(columns = ['fbatchno', 'laner_huaiyun'])
base_df_feature4.fbatchno = laner_huaiyun_batch.批次编号.unique()
base_df_feature4.laner_huaiyun = laner_num_array
base_df_feature4 = base_df_feature4.loc[base_df_feature4.laner_huaiyun > 1]

[ 1  2  1  1  2  1  1  1  1  1  3  2  1  1  1  1  1  3  2  1  1  1  1  2
  1  1  1  1  1  2  2  2  2  3  1  1  1  2  2  1  1  2  1  1  1  1  1  1
  1  1  1  1  1  1  1  2 10 10  2  1  1  2  2  1  1  2  2  2  6  1  1  1
  1  2  2  2  2  1  4  2  2  2  4  2  9  1  1  1  6  1  1  1  1  1  1  2
  1  1  2  2  2  1  1  2  3  2  2  3  2  1  2  1  1  1  1  1  2  2  2  1
  1  4  1  2  1  6  3  2  2  1  8  2  1  2  1  1  2  2  2  1  1  1  1  2
  1  4  2  1  1  1  1  1  1  1  4  1  1  1  1  1  4  1  1  1  1  2  1  1
  1  2  1  1  4  2  2  1  2  1  1  1  2  4  1  2  1  1  1  1  1  1  1  5
  1  1  4  1]


/tmp/ipykernel_15563/2484283823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laner_huaiyun_batch['laner_huaiyun'] = 1


字段5：该场区后备母猪批次驯化隔离期是否<60， 如果有一头母猪隔离期小于60即为是 后备进群到配种是否<60 (从育肥单元转出到配种小于90d)
1. 后备批次，在群状态= 1
2. 该批次在开始时间到转群到待配批次的时间间隔为 < 60 

In [13]:
houbei_gap_sql = '''SELECT  t.*, datediff(t.last_changedate, t.fstarttime) AS day_gap FROM (
SELECT a.fbatchno, a.last_changedate, a.ffieldid, b.fstarttime FROM myana.ybb_batch_retrospect_change_08 a 
inner JOIN 
(SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13908) b 
ON   a.fbatchno = b.fbatchno AND a.ffieldid = b.ffieldid 
WHERE a.to_segmentid = 29535 ) t '''

houbei_gap_df = pd.read_sql(houbei_gap_sql,hiveConn)
houbei_gap_df

,fbatchno,last_changedate,ffieldid,fstarttime,day_gap
0,NLMH00004052530211101002,2022-01-01,15409405,2021-11-01 00:00:01.0,61
1,CX0000410073X211122012,2022-01-22,15389521,2021-11-28 00:00:01.0,55
2,LZ0000420487X211204002,2022-01-22,15417702,2021-12-04 00:00:01.0,49
3,LSX00004193013211222003,2022-01-23,15426357,2021-12-24 00:00:01.0,30
4,FG6104225834211025015,2022-01-19,15257188,2021-10-25 00:00:01.0,86
...,...,...,...,...,...
2910,SL0382C00204100631220114002,2022-01-22,15335524,2022-01-14 00:00:01.0,8
2911,HX00004012536211024004,2022-01-21,15378050,2021-10-24 00:00:01.0,89
2912,XH00004051018211222008,2022-01-21,15388680,2021-12-22 00:00:01.0,30
2913,ZY00004062510211220001,2022-01-23,15470030,2021-12-20 00:00:01.0,34


In [14]:
# 加一步骤，是否大于60做对比，有些批次很多次转群，只要有一次小于60即为是
houbei_gap60 = houbei_gap_df.loc[houbei_gap_df.day_gap < 60, ['fbatchno','ffieldid']].drop_duplicates(subset = ['fbatchno','ffieldid'])
houbei_gap60['day_gap_60'] = 1

字段6：该保育批次是否是大圈配种母猪分娩后代。该批次猪只是否是大圈配种母猪分娩的后代大圈配种：指母猪在生长场配种

In [ ]:
pass #暂时不研究保育批次

字段7：该产房批次是否检出蓝耳病原。一月内产房批次是否检出蓝耳病原，泌乳母猪、哺乳仔猪

In [15]:
claner_changfang_batch = pcr_bi_rawdata.loc[(pcr_bi_rawdata.采样时间 >= '2021-12-15')&(pcr_bi_rawdata.采样时间 <= '2022-01-15')&(pcr_bi_rawdata.批次编号.notnull())&((pcr_bi_rawdata.工段 == '泌乳母猪')|(pcr_bi_rawdata.工段 == '哺乳仔猪'))&(pcr_bi_rawdata.批次编号 != '/')]
claner_changfang_batch['laner_changfang'] = 1
claner_changfang_batch = claner_changfang_batch.loc[:, ['批次编号', 'laner_changfang']]
#laner_changfang_batch.groupby('批次编号').sum()


/tmp/ipykernel_15563/2192551542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claner_changfang_batch['laner_changfang'] = 1


字段8：保育批次猪只产房来源个数。限制工段：保育当前时间该场区保育工段猪只来源涉及产房个数

In [ ]:
pass #

字段9：怀孕批次流产率
1. 在群怀孕批次
2. 一个月内2021-12-15 到2022-01-15 受孕情况和流产率

In [16]:
abortion_rate_sql = ''' SELECT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo WHERE pregndate BETWEEN '2021-12-15' AND '2022-01-15'
GROUP BY foutbatch) A 
INNER JOIN (SELECT fbatchno, ffieldid FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13909) B ON A.foutbatch = B.fbatchno '''

abortion_rate_df = pd.read_sql(abortion_rate_sql,hiveConn)
abortion_rate_df 

,fbatchno,abortion_rate
0,JT0000501333X211124008,0.000000
1,NM7605304099211025023,0.000000
2,ZX590511841X211203009,0.000000
3,FY543223705310914210916003,0.222222
4,BY00005192010211113007,0.000000
...,...,...
6788,MD00005293312210923004,1.000000
6789,WR3805294077211019006,1.000000
6790,GA0000520713X211010006,0.000000
6791,YZXY00005122139211110006,0.000000


In [17]:
other_batch_abortion_sql = '''SELECT DISTINCT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo
GROUP BY foutbatch) A 
INNER JOIN 
(SELECT a.fbatchno, b.fbatchno AS source_fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid, b.ffieldid AS source_fieldid FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid in (13910,21254)) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
WHERE b.fsegmentid = 13909) B
ON A.foutbatch = B.source_fbatchno'''

other_batch_abortion = pd.read_sql(other_batch_abortion_sql,hiveConn)
other_batch_abortion 


,fbatchno,abortion_rate
0,GT00008161330211104017,0.042553
1,KS0000805227412201001,0.094527
2,TX0000603244950210906,0.134921
3,TY0822C00106204359220117018,0.066667
4,TS810836227225210113,0.105263
...,...,...
32171,RP1523C00106271684220115020,0.081197
32172,SX1522C00106105215220102049,0.180851
32173,YP0981C00206283521220106013,0.295652
32174,BY0126C00406037833220107006,0.187500


In [18]:
#怀孕批次流产率-均值
other_batch_abortion_array = other_batch_abortion.groupby(['fbatchno']).mean().abortion_rate.values
print(other_batch_abortion_array) #展示

# 创建新的dataframe 
other_batch_abortion_df = pd.DataFrame(columns = ['fbatchno', 'abortion_rate'])
other_batch_abortion_df.fbatchno = other_batch_abortion.fbatchno.unique()
other_batch_abortion_df.abortion_rate = other_batch_abortion_array
other_batch_abortion_df

[0.47428571 0.21111111 0.21111111 ... 0.17045455 0.14262187 0.10196318]


,fbatchno,abortion_rate
0,GT00008161330211104017,0.474286
1,KS0000805227412201001,0.211111
2,TX0000603244950210906,0.211111
3,TY0822C00106204359220117018,0.211111
4,TS810836227225210113,0.211111
...,...,...
11375,SX71722C00306129655220108044,0.173160
11376,LHK0682050006112910220115001,0.178571
11377,KD0230M0090601101X220109005,0.170455
11378,SX1522C00106105215220102049,0.142622


In [19]:
abortion_df_total =abortion_rate_df.append(other_batch_abortion_df, ignore_index = True)

整合所有在群批次，并且分级。
1. 基础表。所有繁殖场非育肥，非保育，非公猪 工段场区, 在群状态=1
2. 按照字段表现对场区进行划分和批次划分

In [20]:
# 所有批次基础表
batchbase_level_sql = '''SELECT b.*, a.fbatchno, a.fsegmentid FROM  myana.datamining_DWD_BI a 
INNER JOIN (SELECT ffieldid, ffield, fareaid, farea FROM myana.datamining_DIM_AF WHERE ffieldtypename = '繁殖场') b 
ON a.ffieldid = b.ffieldid WHERE a.fsegmentid IN (13910,13909,13905,21254,29535,13908 ) AND a.fstate = 1 '''

batchbase_level_df = pd.read_sql(batchbase_level_sql,hiveConn)
batchbase_level_df

,ffieldid,ffield,fareaid,farea,fbatchno,fsegmentid
0,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00306049461220107010,13910
1,15375746,社旗牧原7场繁殖场,15168980,社旗牧原,SQ1327C00205084430220106011,13909
2,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00301049461220107010,13905
3,15373553,昌图牧原15场繁殖场,15339020,昌图牧原,CT1224C00108130335220106002,21254
4,15442872,东明牧原5场2区繁殖场,15339395,东明牧原,DM1728C00606141528220117030,13910
...,...,...,...,...,...,...
38268,15453142,邛崃牧原3场繁殖场,15448156,邛崃牧原,QL00005091518211219004,13909
38269,15453620,上蔡牧原10场繁殖场,15337687,上蔡牧原,SC1722C01101057367220110001,13905
38270,15472553,射阳牧原1场2区繁殖场,15444027,射阳牧原,SY0924C00305265039220120019,13909
38271,33583,卧龙牧原1场1区繁殖场,13904,卧龙牧原,WL1205264511211222006,13909


In [26]:
# 首先填充某些污染源批次和其字段
batchbase_value_df = batchbase_level_df.copy()
#字段1 
batchbase_value_1 = batchbase_value_df.merge(mom_source_fieldnum_df.loc[:, ['to_fbatchno', 'source_fieldnum']], how = 'left', left_on = 'fbatchno', right_on = 'to_fbatchno')

#字段2 
batchbase_value_2 = batchbase_value_1.merge(laner_houbei_batch.loc[:, ['批次编号', 'laner_houbei']], how = 'left', left_on = 'fbatchno', right_on = '批次编号')

#字段3
batchbase_value_3 = batchbase_value_2.merge(dajuan_batch_total.loc[:, ['fbatchno', '大圈配种怀孕批次']], how = 'left', on = 'fbatchno')

#字段4
batchbase_value_4 = batchbase_value_3.merge(base_df_feature4, how = 'left', on = 'fbatchno')

#字段5
batchbase_value_5 = batchbase_value_4.merge(houbei_gap60.loc[:, ['fbatchno', 'day_gap_60']], how = 'left', on = 'fbatchno')

# 字段7
batchbase_value_7 = batchbase_value_5.merge(claner_changfang_batch.loc[:, ['批次编号', 'laner_changfang']], how = 'left', left_on = 'fbatchno', right_on = '批次编号')

#字段9 
batchbase_value_9 = batchbase_value_7.merge(abortion_df_total.loc[:, ['fbatchno', 'abortion_rate']], how = 'left', on = 'fbatchno')


In [27]:
batchbase_value_9

,ffieldid,ffield,fareaid,farea,fbatchno,fsegmentid,to_fbatchno,source_fieldnum,批次编号_x,laner_houbei,大圈配种怀孕批次,laner_huaiyun,day_gap_60,批次编号_y,laner_changfang,abortion_rate
0,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00306049461220107010,13910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15375746,社旗牧原7场繁殖场,15168980,社旗牧原,SQ1327C00205084430220106011,13909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00301049461220107010,13905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15373553,昌图牧原15场繁殖场,15339020,昌图牧原,CT1224C00108130335220106002,21254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15442872,东明牧原5场2区繁殖场,15339395,东明牧原,DM1728C00606141528220117030,13910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38512,15166930,扶沟牧原1场繁殖场,15116388,扶沟牧原,FG1621010005194616220121009,13909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38513,15352723,凤台牧原2场2区繁殖场,15339597,凤台牧原,FT0421C0020101563X220118020,13905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38514,15341968,牡丹牧原2场繁殖场,15341960,牡丹牧原,MD1702C00205165218220121017,13909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38515,15390579,石林牧原1场繁殖场,15390573,石林牧原,SL0126D00101081417220121010,13905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


划分整活. 需要在基础等级表给每一个批次划分等级
1. 创建两个新列：等级和划分原因
2. 对于每一个批次，在字段值里找对应的等级划分逻辑的等级。并且标上所有的划分原因

In [28]:
batchbase_value = batchbase_value_9.loc[:, ['ffieldid', 'ffield', 'fareaid', 'farea', 'fbatchno', 'fsegmentid', 'source_fieldnum', 'laner_houbei', '大圈配种怀孕批次', 'laner_huaiyun', 
                                           'day_gap_60', 'laner_changfang', 'abortion_rate']]

batchbase_value

,ffieldid,ffield,fareaid,farea,fbatchno,fsegmentid,source_fieldnum,laner_houbei,大圈配种怀孕批次,laner_huaiyun,day_gap_60,laner_changfang,abortion_rate
0,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00306049461220107010,13910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15375746,社旗牧原7场繁殖场,15168980,社旗牧原,SQ1327C00205084430220106011,13909,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00301049461220107010,13905,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15373553,昌图牧原15场繁殖场,15339020,昌图牧原,CT1224C00108130335220106002,21254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15442872,东明牧原5场2区繁殖场,15339395,东明牧原,DM1728C00606141528220117030,13910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38512,15166930,扶沟牧原1场繁殖场,15116388,扶沟牧原,FG1621010005194616220121009,13909,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38513,15352723,凤台牧原2场2区繁殖场,15339597,凤台牧原,FT0421C0020101563X220118020,13905,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38514,15341968,牡丹牧原2场繁殖场,15341960,牡丹牧原,MD1702C00205165218220121017,13909,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38515,15390579,石林牧原1场繁殖场,15390573,石林牧原,SL0126D00101081417220121010,13905,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#找到污染源
pollution_batch = batchbase_value.dropna(subset = ['source_fieldnum', 'laner_houbei', '大圈配种怀孕批次', 'laner_huaiyun', 'day_gap_60', 'laner_changfang', 'abortion_rate'], how = 'all').index 
#对污染源添加字段 pollution_batch
batchbase_value.loc[pollution_batch , '是否污染源'] = '是'

In [30]:
#划分等级, 1,2,3等级分别对应 高风险，中风险，低风险
batchbase_value_copy = batchbase_value.copy()

#该场区后备母猪来源涉及场区个数>1， 所有场区高风险
highrisk_field_1 = list(batchbase_value.loc[batchbase_value.source_fieldnum > 1].ffieldid.unique()) #高风险场区列表
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_1), ['risk_level1', 'reason1']] = 1, '该场区后备母猪来源涉及场区个数>1'

#该场区后备猪进群前检出蓝耳抗原， 所有场区高风险
highrisk_field_2 = list(batchbase_value.loc[batchbase_value.laner_houbei.notnull()].ffieldid.unique()) 
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_2), ['risk_level2', 'reason2']] = 1, '该场区后备猪进群前检出蓝耳抗原'

#一胎怀孕母猪是大圈配种的批次, 仅指这个批次
batchbase_value_copy.loc[batchbase_value.大圈配种怀孕批次.notnull(), ['risk_level3', 'reason3']] = 1, '该批次是一胎怀孕母猪是大圈配种的批次'

#该场区怀孕批次检出蓝耳抗原个数>1， 所有场区中风险
highrisk_field_4 = list(batchbase_value.loc[batchbase_value.laner_huaiyun.notnull()].ffieldid.unique())
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_4), ['risk_level4', 'reason4']] = 2, '该场区怀孕批次检出蓝耳抗原个数>1'

#该场区后备母猪隔离期<60， 所有场区中风险
highrisk_field_5 = list(batchbase_value.loc[batchbase_value.day_gap_60.notnull()].ffieldid.unique())
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_5), ['risk_level5', 'reason5']] = 2, '该场区后备母猪隔离期<60'

#该批次是大圈配种母猪分娩后代

#该产房批次病原检出批次个数>1
batchbase_value_copy.loc[batchbase_value.laner_changfang.notnull(), ['risk_level6', 'reason6']] = 2, '该产房批次病原检出批次个数>1'

#保育接猪混群单元>=2(产房单元>=2)

#怀孕批次流产率>3% = 高风险, 1.5%<怀孕批次流产率≤3% = 中风险, 怀孕批次流产率≤1.5% = 低风险
batchbase_value_copy.loc[batchbase_value.abortion_rate > 0.03, ['risk_level7', 'reason7']] = 1, '怀孕批次流产率>3%'
batchbase_value_copy.loc[(batchbase_value.abortion_rate <= 0.03)&(batchbase_value.abortion_rate > 0.015), ['risk_level7', 'reason7']] = 2, '1.5%<怀孕批次流产率≤3%'
batchbase_value_copy.loc[batchbase_value.abortion_rate <= 0.015, ['risk_level7', 'reason7']] = 3, '怀孕批次流产率≤1.5%'

#该场区后备母猪来源涉及场区个数 = 1 低风险
highrisk_field_6 = list(batchbase_value.loc[batchbase_value.source_fieldnum == 1].ffieldid.unique()) #风险场区列表
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_6), ['risk_level8', 'reason8']] = 3, '该场区后备母猪来源涉及场区个数=1'




In [31]:
#risk_level所有数值权为空则为 ’无法判断“等级
batchbase_value_nonnull = batchbase_value_copy.dropna(subset = ['risk_level1','risk_level2', 'risk_level3', 'risk_level4', 'risk_level5', 'risk_level6', 'risk_level7', 'risk_level8'], how = 'all')

# 每一行第一个非空数字即为风险等级
risk_values = batchbase_value_nonnull.loc[:, ['risk_level1','risk_level2', 'risk_level3', 'risk_level4', 'risk_level5', 'risk_level6', 'risk_level7', 'risk_level8']].values 
batchbase_value_nonnull["risk_level_no"] = [risk_values[x][np.where(~np.isnan(risk_values[x]))][0] for x in range(len(batchbase_value_nonnull))]
risk_dict = {1:'高风险', 2:'中风险', 3:'低风险'} #风险等级字典
batchbase_value_nonnull['风险等级'] = batchbase_value_nonnull.risk_level_no.replace(risk_dict)
#填充到原来的基础表，index不变
batchbase_value_copy['风险等级'] = batchbase_value_nonnull['风险等级']
#其他没有划分等级的批次, 填充为”无法判断“
batchbase_value_copy.风险等级 = batchbase_value_copy.风险等级.fillna('无法判断')

batchbase_value_copy.风险等级

/tmp/ipykernel_15201/1085136941.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batchbase_value_nonnull["risk_level_no"] = [risk_values[x][np.where(~np.isnan(risk_values[x]))][0] for x in range(len(batchbase_value_nonnull))]
/tmp/ipykernel_15201/1085136941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batchbase_value_nonnull['风险等级'] = batchbase_value_nonnull.risk_level_no.replace(risk_dict)


0        低风险
1        中风险
2        低风险
3        中风险
4        高风险
        ... 
38512    高风险
38513    高风险
38514    高风险
38515    高风险
38516    高风险
Name: 风险等级, Length: 38517, dtype: object

等级划分原因罗列, 缺点：某一列如果为空，另一列不为空，则合并结果是为空，所以在合并之前需要对NA进行预处理，替换或者删除。

In [32]:
reason_list = ['reason1', 'reason2', 'reason3', 'reason4', 'reason5', 'reason6', 'reason7', 'reason8']
batchbase_value_copy['划分原因'] = batchbase_value_copy.loc[:,reason_list].apply(lambda x: ','.join(x.dropna()), axis = 1)


更新到比较干净的原始表格，只添加两个字段：风险等级和划分逻辑

In [33]:
batchbase_value.loc[:, ['风险等级', '划分原因']] = batchbase_value_copy.loc[:, ['风险等级', '划分原因']]

In [34]:
# 工段
segment_dict = {13910:'泌乳母猪', 13909:'怀孕', 13905:'哺乳仔猪',21254: '空怀', 29535:'待配', 13908:'后备'}

batchbase_value.fsegmentid = batchbase_value.fsegmentid.replace(segment_dict)
batchbase_value.是否污染源 = batchbase_value.是否污染源.fillna('否')

In [35]:
batchbase_value.head()

,ffieldid,ffield,fareaid,farea,fbatchno,fsegmentid,source_fieldnum,laner_houbei,大圈配种怀孕批次,laner_huaiyun,day_gap_60,laner_changfang,abortion_rate,是否污染源,风险等级,划分原因
0,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00306049461220107010,泌乳母猪,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,低风险,该场区后备母猪来源涉及场区个数=1
1,15375746,社旗牧原7场繁殖场,15168980,社旗牧原,SQ1327C00205084430220106011,怀孕,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,中风险,"该场区怀孕批次检出蓝耳抗原个数>1,该场区后备母猪隔离期<60,该场区后备母猪来源涉及场区个数=1"
2,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,SQ1327C00301049461220107010,哺乳仔猪,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,低风险,该场区后备母猪来源涉及场区个数=1
3,15373553,昌图牧原15场繁殖场,15339020,昌图牧原,CT1224C00108130335220106002,空怀,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,中风险,"该场区后备母猪隔离期<60,该场区后备母猪来源涉及场区个数=1"
4,15442872,东明牧原5场2区繁殖场,15339395,东明牧原,DM1728C00606141528220117030,泌乳母猪,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,高风险,"该场区后备母猪来源涉及场区个数>1,该场区后备母猪来源涉及场区个数=1"


In [36]:
batchbase_value.loc[batchbase_value.ffield == '卧龙牧原15场繁殖场'].风险等级.value_counts()

低风险    68
高风险    29
Name: 风险等级, dtype: int64

# 保存表格

In [37]:
save_batchvalue_df = batchbase_value.rename(columns = {'ffieldid':'所在场区ID', 'ffield':'所在场区名', 'fareaid':'所在子公司id','farea':'所在子公司名', 
                                                       'fbatchno':'批次号', 'fsegmentid':'工段','source_fieldnum':'该场区母猪来源涉及场区个数', 'laner_houbei':'该场区后备猪批次进群前是否检出蓝耳抗原', 
                                                        'laner_huaiyun':'该场区怀孕批次检出蓝耳抗原个数', 
                                                       'day_gap_60':'该场区后备母猪批次驯化隔离期是否<60', 'laner_changfang':'该场区产房批次是否检出蓝耳病原','abortion_rate':'怀孕批次流产率'})

In [143]:
#save_batchvalue_df.loc[save_batchvalue_df.所在场区名 == '卧龙牧原15场繁殖场'].to_csv('卧龙牧原15场繁殖场_蓝耳.csv', index = False)

In [38]:
save_batchvalue_df.风险等级.value_counts()

高风险     27080
中风险      7923
低风险      3410
无法判断      104
Name: 风险等级, dtype: int64

In [ ]:
sns.countplot(save_batchvalue_df.风险等级)

# 其他场区取数
1. 一胎怀孕母猪批次配种的场区类型。添加其他保育和育肥批次的母猪
2. 该保育批次是否是大圈配种母猪分娩后代。同样追溯生身母猪
3. 保育批次猪只产房来源个数
4. 怀孕批次流产率。 同样需要添加保育和育肥段，需要追溯到其母猪

In [21]:
#一胎怀孕母猪批次配种的场区类型
# 保育---该保育批次是否是大圈配种母猪分娩后代
baoyu_mom_batch_sql = '''SELECT DISTINCT t.fbatchno, t.配种场区id FROM (
SELECT DISTINCT  a.fbatchno, b.fsegmentid AS source_segmentid , b.to_segmentid, 
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid, f.ffieldtypename AS 配种场区id
FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13906) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno
LEFT JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno 
INNER JOIN  myana.datamining_DIM_AF f ON bbb.ffieldid = f.ffieldid ) t 
WHERE t.source_segmentid = 13905 AND t.to_segmentid = 13906 AND t.source2_segmentid = 13910 AND t.source3_segmentid = 13909 '''

baoyu_mom_batch = pd.read_sql(baoyu_mom_batch_sql,hiveConn)

# 添加字段：是否是大圈配种怀孕批次
dajuan_batch4 = baoyu_mom_batch.loc[~(baoyu_mom_batch.配种场区id == '繁殖场')]
dajuan_batch4['大圈配种怀孕批次']='是'
dajuan_batch4 = dajuan_batch4.loc[:, ['fbatchno', '配种场区id', '大圈配种怀孕批次']].drop_duplicates()
dajuan_batch4

/tmp/ipykernel_15563/1494520690.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dajuan_batch4['大圈配种怀孕批次']='是'


,fbatchno,配种场区id,大圈配种怀孕批次
10,RZ00002164775211117030,生长场,是
22,LHK2502201020211205016,生长场,是
23,SY1403D00102257492220103011,生长场,是
28,LZ00002155474211218030,生长场,是
32,CX2902045435211113029,生长场,是
...,...,...,...
17630,LHK9402216316211223017,生长场,是
17635,LHK0682D00902033516220117002,生长场,是
17641,QX00002183016211228020,生长场,是
17644,WL1502201597211210065,生长场,是


In [46]:
#保育批次是否是大圈配种母猪分娩后代 = dajuan_batch4
baoyu_mom_dajuan = dajuan_batch4.rename(columns = {'大圈配种怀孕批次':'保育批次是否是大圈配种母猪分娩后代'})
baoyu_mom_dajuan

,fbatchno,配种场区id,保育批次是否是大圈配种母猪分娩后代
1,LZ00002155474211218030,生长场,是
8,CX8202300504211209019,生长场,是
13,NXZHT1391M01302040355220107030,生长场,是
23,SX1324D00402030616220112009,生长场,是
33,KS00002233622211208009,综合场,是
...,...,...,...
17636,MC00002024946211103024,综合场,是
17642,NXZHT00002103324211220027,综合场,是
17644,ZX2202290611211224035,综合场,是
17650,NXZHT1391M0210213064X220109009,综合场,是


In [22]:
yufei_mom_batch_sql = '''SELECT DISTINCT  t.fbatchno, t.配种场区id FROM (
SELECT DISTINCT  a.fbatchno, b.fsegmentid AS source_segmentid , b.to_segmentid, 
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid, bbbb.fsegmentid AS source4_segmentid, f.ffieldtypename AS 配种场区id
FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13907) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno
LEFT JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bbbb ON bbb.fbatchno = bbbb.to_fbatchno 
INNER JOIN  myana.datamining_DIM_AF f ON bbbb.ffieldid = f.ffieldid ) t 
WHERE t.source_segmentid = 13906 AND t.to_segmentid = 13907 AND t.source2_segmentid = 13905 AND t.source3_segmentid = 13910 AND t.source4_segmentid = 13909 '''

yufei_mom_batch = pd.read_sql(yufei_mom_batch_sql,hiveConn)

# 添加字段：是否是大圈配种怀孕批次
dajuan_batch5 = yufei_mom_batch.loc[~(yufei_mom_batch.配种场区id == '繁殖场')]
dajuan_batch5['大圈配种怀孕批次']='是'
dajuan_batch5 = dajuan_batch5.loc[:, ['fbatchno', '配种场区id', '大圈配种怀孕批次']].drop_duplicates()
dajuan_batch5

/tmp/ipykernel_15563/1185346798.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dajuan_batch5['大圈配种怀孕批次']='是'


,fbatchno,配种场区id,大圈配种怀孕批次
2,LHK5903082413210929004,综合场,是
6,XJ0825D00103043416220118048,生长场,是
7,LX0000315305X211009027,综合场,是
11,ML0000305282500210829,综合场,是
15,JZ1181D00403063118220102125,生长场,是
...,...,...,...
34117,ZX6403154015211229019,综合场,是
34119,JP1322003103142518220103054,生长场,是
34127,LJ0221M00303130812220101076,综合场,是
34135,FT0421D00703244618220116039,生长场,是


In [23]:
#保育批次猪只产房来源个数
baoyu_chanfang_num_sql = '''SELECT t.fbatchno, count(t.source_batch) AS source_chanfang_num FROM (
SELECT a.fbatchno, b.fbatchno AS source_batch FROM (
SELECT fbatchno, fsegmentid FROM  myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13906) a
LEFT JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
WHERE b.fsegmentid = 13905) t GROUP BY t.fbatchno '''

baoyu_chanfang_num = pd.read_sql(baoyu_chanfang_num_sql,hiveConn)
baoyu_chanfang_num

,fbatchno,source_chanfang_num
0,HZ00002071816211225008,1
1,FX3702161287211207024,1
2,TY00002056414211223008,1
3,SC00002161360211108013,1
4,HS00002173329211219029,1
...,...,...
15787,SS00002202528211216006,1
15788,SL00002250324211226001,3
15789,DZ138102010225795X220106002,1
15790,DL0523050002014269220114010,1


In [39]:
# 保育和育肥段怀孕流产率
baoyu_mom_abortion_sql = '''SELECT T.fbatchno, AVG(T.abortion_rate) AS abortion_rate FROM (
SELECT DISTINCT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo
GROUP BY foutbatch) A 
INNER JOIN (
SELECT DISTINCT t.fbatchno, t.怀孕批次号 FROM (
SELECT a.fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid,
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid , bbb.fbatchno AS 怀孕批次号  FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13906) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno) t 
WHERE t.source_segmentid = 13905 AND t.to_segmentid = 13906 AND t.source2_segmentid = 13910 AND t.source3_segmentid = 13909) B 
ON A.foutbatch = B.怀孕批次号) T 
GROUP BY T.fbatchno '''

baoyu_mom_abortion = pd.read_sql(baoyu_mom_abortion_sql,hiveConn)

# 育肥段
yufei_mom_abortion_sql = '''SELECT T.fbatchno, avg(T.abortion_rate) AS abortion_rate FROM (
SELECT DISTINCT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo
GROUP BY foutbatch) A 
INNER JOIN (
SELECT a.fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid,
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid, bbbb.fsegmentid AS source4_segmentid, bbbb.fbatchno AS 怀孕批次号 FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13907) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno
INNER JOIN myana.ybb_batch_retrospect_change_08 bbbb ON bbb.fbatchno = bbbb.to_fbatchno ) B 
ON A.foutbatch = B.怀孕批次号) T 
GROUP BY T.fbatchno '''

yufei_mom_abortion = pd.read_sql(yufei_mom_abortion_sql,hiveConn)


In [40]:
yufei_mom_abortion

,fbatchno,abortion_rate
0,SX00003155414211216015,0.176069
1,XH120330691X211122023,0.236135
2,YP00003280119211007038,0.158410
3,BY00003254014211003137,0.133771
4,AH5703203759211203055,0.287897
...,...,...
28048,CX7303010896211112016,0.164835
28049,QX0903023017211217031,0.046595
28050,YP00003023213211109009,0.301887
28051,SC00003107018211206007,0.146745


# 其他场区汇总，9个字段

In [148]:
# 字段
# 脚本
# 所有批次基础表
hiveConn = hive.Connection(host='10.106.20.15', port=10000, username='szchenye', password='szcy230#',
                                               database='mydw', auth='CUSTOM')

batchbase_level_sql = '''SELECT b.*, a.fbatchno, a.fsegmentid FROM  myana.datamining_DWD_BI a 
INNER JOIN (SELECT ffieldid, ffield, fareaid, farea, fregionname, fbigregionname FROM myana.datamining_DIM_AF) b 
ON a.ffieldid = b.ffieldid WHERE a.fstate = 1 '''

batchbase_level_df = pd.read_sql(batchbase_level_sql,hiveConn)


#该场区母猪来源涉及场区个数
mom_source_fieldnum_sql = '''SELECT t.to_fbatchno,  t.to_ffieldid, t.to_segmentid, count(DISTINCT t.ffieldid) AS source_fieldnum FROM (
SELECT fbatchno, to_fbatchno, ffieldid, to_ffieldid, to_segmentid FROM myana.ybb_batch_retrospect_change_08
WHERE to_segmentid IN (13910, 13909, 21254,29535, 13908 )
AND last_changedate BETWEEN '2021-12-15' AND '2022-01-15') t 
GROUP BY t.to_fbatchno, t.to_ffieldid , t.to_segmentid '''

mom_source_fieldnum_df = pd.read_sql(mom_source_fieldnum_sql,hiveConn)

#该场区后备猪批次进群前是否检出蓝耳抗原
#筛选出后备批次在这一个月内检出蓝耳的场区
laner_houbei_batch = pcr_bi_rawdata.loc[(pcr_bi_rawdata.采样时间 >= '2021-12-15')&(pcr_bi_rawdata.采样时间 <= '2022-01-15')&(pcr_bi_rawdata.工段 == '后备')&(pcr_bi_rawdata.送检目的 == '转群监控')]
laner_houbei_batch['laner_houbei'] = 1
laner_houbei_batch = laner_houbei_batch.drop_duplicates(subset = ['批次编号'])

#一胎怀孕母猪批次（后备）配种的场区类型
houbei_field_sql = '''
SELECT DISTINCT A.fbatchno, B.ffieldtypename AS 配种场区id FROM (
SELECT a.fbatchno, a.ffieldid, a.to_ffieldid FROM myana.ybb_batch_retrospect_change_08 a 
inner JOIN 
(SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13908) b 
ON   a.fbatchno = b.fbatchno AND a.ffieldid = b.ffieldid 
WHERE a.to_segmentid = 29535 ) A 
LEFT JOIN  myana.datamining_DIM_AF B  ON A.to_ffieldid = B.ffieldid '''

houbei_field_df = pd.read_sql(houbei_field_sql,hiveConn)
#大圈的怀孕批次
dajuan_batch = houbei_field_df.loc[~(houbei_field_df.配种场区id == '繁殖场')]
dajuan_batch['大圈配种怀孕批次']='是'
dajuan_batch = dajuan_batch.loc[:, ['fbatchno', '大圈配种怀孕批次']].drop_duplicates()

# 待配和怀孕批次
daipei_batch_sql = ''' SELECT DISTINCT A.fbatchno, B.ffieldtypename AS 配种场区id FROM (
SELECT a.fbatchno, b.fbatchno AS source_fbatchno , b.fsegmentid, b.to_segmentid ,b.to_ffieldid AS 待配场区ID FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid in (13909,29535) ) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno WHERE b.fsegmentid = 13908) A 
LEFT JOIN myana.datamining_DIM_AF B  ON A.待配场区ID = B.ffieldid '''
daipei_batch_df = pd.read_sql(daipei_batch_sql,hiveConn)

#大圈的怀孕批次
dajuan_batch2 = daipei_batch_df.loc[~(daipei_batch_df.配种场区id == '繁殖场')]
dajuan_batch2['大圈配种怀孕批次']='是'
dajuan_batch2 = dajuan_batch2.loc[:, ['fbatchno', '大圈配种怀孕批次']].drop_duplicates()

# 其他母猪批次，需要追述到第一次怀孕的地点（也就是13908后备 转到29535待配时，待配的场区类型）
other_mom_batch_sql = '''SELECT DISTINCT abc.fbatchno, f.ffieldtypename AS 配种场区id FROM (
SELECT DISTINCT t.fbatchno, t.source2_fieldid AS 待配场区id FROM (
SELECT aaa.*, bbb.fbatchno AS source3_fbatchno,bbb.fsegmentid AS source3_segmentid, bbb.ffieldid AS source3_fieldid FROM(
SELECT aa.*, bb.fbatchno AS source_source_fbatchno, bb.fsegmentid AS source2_segmentid, bb.ffieldid AS source2_fieldid FROM (
SELECT a.fbatchno, b.fbatchno AS source_fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid, b.ffieldid AS source_fieldid FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid in (13910,21254)) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno ) aa 
LEFT JOIN  myana.ybb_batch_retrospect_change_08 bb ON aa.source_fbatchno = bb.to_fbatchno ) aaa 
LEFT JOIN myana.ybb_batch_retrospect_change_08 bbb ON aaa.source_source_fbatchno = bbb.to_fbatchno ) t 
WHERE t.source2_segmentid = 29535 AND t.source3_segmentid = 13908) abc 
LEFT JOIN  myana.datamining_DIM_AF f ON abc.待配场区id = f.ffieldid '''

other_mom_batch = pd.read_sql(other_mom_batch_sql,hiveConn)
dajuan_batch3 = other_mom_batch.loc[~(other_mom_batch.配种场区id == '繁殖场')]
dajuan_batch3['大圈配种怀孕批次']='是'
dajuan_batch3 = dajuan_batch3.loc[:, ['fbatchno', '大圈配种怀孕批次']].drop_duplicates()

#一胎怀孕母猪批次配种的场区类型
# 保育---该保育批次是否是大圈配种母猪分娩后代
baoyu_mom_batch_sql = '''SELECT DISTINCT t.fbatchno, t.配种场区id FROM (
SELECT DISTINCT  a.fbatchno, b.fsegmentid AS source_segmentid , b.to_segmentid, 
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid, f.ffieldtypename AS 配种场区id
FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13906) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno
LEFT JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno 
INNER JOIN  myana.datamining_DIM_AF f ON bbb.ffieldid = f.ffieldid ) t 
WHERE t.source_segmentid = 13905 AND t.to_segmentid = 13906 AND t.source2_segmentid = 13910 AND t.source3_segmentid = 13909 '''

baoyu_mom_batch = pd.read_sql(baoyu_mom_batch_sql,hiveConn)

# 添加字段：是否是大圈配种怀孕批次
dajuan_batch4 = baoyu_mom_batch.loc[~(baoyu_mom_batch.配种场区id == '繁殖场')]
dajuan_batch4['大圈配种怀孕批次']='是'
dajuan_batch4 = dajuan_batch4.loc[:, ['fbatchno', '大圈配种怀孕批次']].drop_duplicates()

yufei_mom_batch_sql = '''SELECT DISTINCT  t.fbatchno, t.配种场区id FROM (
SELECT DISTINCT  a.fbatchno, b.fsegmentid AS source_segmentid , b.to_segmentid, 
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid, bbbb.fsegmentid AS source4_segmentid, f.ffieldtypename AS 配种场区id
FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13907) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno
LEFT JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bbbb ON bbb.fbatchno = bbbb.to_fbatchno 
INNER JOIN  myana.datamining_DIM_AF f ON bbbb.ffieldid = f.ffieldid ) t 
WHERE t.source_segmentid = 13906 AND t.to_segmentid = 13907 AND t.source2_segmentid = 13905 AND t.source3_segmentid = 13910 AND t.source4_segmentid = 13909 '''

yufei_mom_batch = pd.read_sql(yufei_mom_batch_sql,hiveConn)

# 添加字段：是否是大圈配种怀孕批次
dajuan_batch5 = yufei_mom_batch.loc[~(yufei_mom_batch.配种场区id == '繁殖场')]
dajuan_batch5['大圈配种怀孕批次']='是'
dajuan_batch5 = dajuan_batch5.loc[:, ['fbatchno', '大圈配种怀孕批次']].drop_duplicates()

#总和所有工段
dajuan_batch_total = dajuan_batch.append(dajuan_batch2, ignore_index = True).append(dajuan_batch3, ignore_index = True).append(dajuan_batch4, ignore_index = True).append(dajuan_batch5, ignore_index = True)


/tmp/ipykernel_15563/2688581759.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laner_houbei_batch['laner_houbei'] = 1
/tmp/ipykernel_15563/2688581759.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dajuan_batch['大圈配种怀孕批次']='是'
/tmp/ipykernel_15563/2688581759.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [149]:
laner_huaiyun_batch = pcr_bi_rawdata.loc[(pcr_bi_rawdata.采样时间 >= '2021-12-15')&(pcr_bi_rawdata.采样时间 <= '2022-01-15')&(pcr_bi_rawdata.批次编号.notnull())&(pcr_bi_rawdata.工段 == '怀孕')&(pcr_bi_rawdata.批次编号 != '/')]
laner_huaiyun_batch['laner_huaiyun'] = 1

 #蓝耳抗原数量
laner_num_array = laner_huaiyun_batch.loc[:, ['批次编号', 'laner_huaiyun']].groupby(['批次编号']).sum().laner_huaiyun.values
# 创建新的dataframe 
base_df_feature4 = pd.DataFrame(columns = ['fbatchno', 'laner_huaiyun'])
base_df_feature4.fbatchno = laner_huaiyun_batch.批次编号.unique()
base_df_feature4.laner_huaiyun = laner_num_array
base_df_feature4 = base_df_feature4.loc[base_df_feature4.laner_huaiyun > 1]

#该场区后备母猪批次驯化隔离期是否<60
houbei_gap_sql = '''SELECT  t.*, datediff(t.last_changedate, t.fstarttime) AS day_gap FROM (
SELECT a.fbatchno, a.last_changedate, a.ffieldid, b.fstarttime FROM myana.ybb_batch_retrospect_change_08 a 
inner JOIN 
(SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13908) b 
ON   a.fbatchno = b.fbatchno AND a.ffieldid = b.ffieldid 
WHERE a.to_segmentid = 29535 ) t '''

houbei_gap_df = pd.read_sql(houbei_gap_sql,hiveConn)

# 加一步骤，是否大于60做对比，有些批次很多次转群，只要有一次小于60即为是
houbei_gap60 = houbei_gap_df.loc[houbei_gap_df.day_gap < 60, ['fbatchno','ffieldid']].drop_duplicates(subset = ['fbatchno','ffieldid'])
houbei_gap60['day_gap_60'] = 1

#该保育批次是否是大圈配种母猪分娩后代 = dajuan_batch4
baoyu_mom_dajuan = dajuan_batch4.rename(columns = {'大圈配种怀孕批次':'保育批次是否是大圈配种母猪分娩后代'})

#该场区产房批次是否检出蓝耳病原
laner_changfang_batch = pcr_bi_rawdata.loc[(pcr_bi_rawdata.采样时间 >= '2021-12-15')&(pcr_bi_rawdata.采样时间 <= '2022-01-15')&(pcr_bi_rawdata.批次编号.notnull())&((pcr_bi_rawdata.工段 == '泌乳母猪')|(pcr_bi_rawdata.工段 == '哺乳仔猪'))&(pcr_bi_rawdata.批次编号 != '/')]
laner_changfang_batch['laner_changfang'] = 1
laner_changfang_batch = laner_changfang_batch.loc[:, ['批次编号', 'laner_changfang']].groupby('批次编号').sum().reset_index()

#保育批次猪只产房来源个数
baoyu_chanfang_num_sql = '''SELECT t.fbatchno, count(t.source_batch) AS source_chanfang_num FROM (
SELECT a.fbatchno, b.fbatchno AS source_batch FROM (
SELECT fbatchno, fsegmentid FROM  myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13906) a
LEFT JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
WHERE b.fsegmentid = 13905) t GROUP BY t.fbatchno '''

baoyu_chanfang_num = pd.read_sql(baoyu_chanfang_num_sql,hiveConn)

#怀孕批次流产率
abortion_rate_sql = ''' SELECT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo WHERE pregndate BETWEEN '2021-12-15' AND '2022-01-15'
GROUP BY foutbatch) A 
INNER JOIN (SELECT fbatchno, ffieldid FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13909) B ON A.foutbatch = B.fbatchno '''

abortion_rate_df = pd.read_sql(abortion_rate_sql,hiveConn)

other_batch_abortion_sql = '''SELECT T.fbatchno, avg(T.abortion_rate) AS abortion_rate FROM (
SELECT DISTINCT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo
GROUP BY foutbatch) A 
INNER JOIN 
(SELECT a.fbatchno, b.fbatchno AS source_fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid, b.ffieldid AS source_fieldid FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid in (13910,21254)) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
WHERE b.fsegmentid = 13909) B
ON A.foutbatch = B.source_fbatchno) T GROUP BY T.fbatchno'''
other_batch_abortion = pd.read_sql(other_batch_abortion_sql,hiveConn)

# 保育和育肥段怀孕流产率
baoyu_mom_abortion_sql = '''SELECT T.fbatchno, AVG(T.abortion_rate) AS abortion_rate FROM (
SELECT DISTINCT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo
GROUP BY foutbatch) A 
INNER JOIN (
SELECT DISTINCT t.fbatchno, t.怀孕批次号 FROM (
SELECT a.fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid,
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid , bbb.fbatchno AS 怀孕批次号  FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13906) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno) t 
WHERE t.source_segmentid = 13905 AND t.to_segmentid = 13906 AND t.source2_segmentid = 13910 AND t.source3_segmentid = 13909) B 
ON A.foutbatch = B.怀孕批次号) T 
GROUP BY T.fbatchno '''

baoyu_mom_abortion = pd.read_sql(baoyu_mom_abortion_sql,hiveConn)

# 育肥段
yufei_mom_abortion_sql = '''SELECT T.fbatchno, avg(T.abortion_rate) AS abortion_rate FROM (
SELECT DISTINCT B.fbatchno, A.abortion_rate FROM (
SELECT foutbatch,
SUM(CASE WHEN pregnresult in ('流产') THEN 1 ELSE 0 END)/SUM(CASE WHEN pregnresult in ('流产','返情','检空','空怀') THEN 1 ELSE 0 END) AS abortion_rate
FROM mydw.source_my_sc_pregnancyinfo
GROUP BY foutbatch) A 
INNER JOIN (
SELECT a.fbatchno , b.fsegmentid AS source_segmentid , b.to_segmentid,
bb.fsegmentid AS source2_segmentid, bbb.fsegmentid AS source3_segmentid, bbbb.fsegmentid AS source4_segmentid, bbbb.fbatchno AS 怀孕批次号 FROM (
SELECT * FROM myana.datamining_DWD_BI WHERE fstate = 1 AND fsegmentid = 13907) a 
INNER JOIN myana.ybb_batch_retrospect_change_08 b ON a.fbatchno = b.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bb ON b.fbatchno = bb.to_fbatchno 
INNER JOIN myana.ybb_batch_retrospect_change_08 bbb ON bb.fbatchno = bbb.to_fbatchno
INNER JOIN myana.ybb_batch_retrospect_change_08 bbbb ON bbb.fbatchno = bbbb.to_fbatchno ) B 
ON A.foutbatch = B.怀孕批次号) T 
GROUP BY T.fbatchno '''

yufei_mom_abortion = pd.read_sql(yufei_mom_abortion_sql,hiveConn)

#所有工段的怀孕流产率

abortion_df_total =abortion_rate_df.append(other_batch_abortion_df, ignore_index = True).append(baoyu_mom_abortion, ignore_index = True).append(yufei_mom_abortion, ignore_index = True)

/tmp/ipykernel_15563/4289163477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laner_huaiyun_batch['laner_huaiyun'] = 1
/tmp/ipykernel_15563/4289163477.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laner_changfang_batch['laner_changfang'] = 1


In [164]:
# 首先填充某些污染源批次和其字段
batchbase_value_df = batchbase_level_df.copy()
#字段1 
batchbase_value_1 = batchbase_value_df.merge(mom_source_fieldnum_df.loc[:, ['to_fbatchno', 'source_fieldnum']], how = 'left', left_on = 'fbatchno', right_on = 'to_fbatchno')

#字段2 
batchbase_value_2 = batchbase_value_1.merge(laner_houbei_batch.loc[:, ['批次编号', 'laner_houbei']], how = 'left', left_on = 'fbatchno', right_on = '批次编号')

#字段3
batchbase_value_3 = batchbase_value_2.merge(dajuan_batch_total.loc[:, ['fbatchno', '大圈配种怀孕批次']], how = 'left', on = 'fbatchno')

#字段4
batchbase_value_4 = batchbase_value_3.merge(base_df_feature4, how = 'left', on = 'fbatchno')

#字段5
batchbase_value_5 = batchbase_value_4.merge(houbei_gap60.loc[:, ['fbatchno', 'day_gap_60']], how = 'left', on = 'fbatchno')

#字段6 
batchbase_value_6 = batchbase_value_5.merge(baoyu_mom_dajuan.loc[:, ['fbatchno', '保育批次是否是大圈配种母猪分娩后代']], how = 'left', on = 'fbatchno')

# 字段7
batchbase_value_7 = batchbase_value_6.merge(laner_changfang_batch.loc[:, ['批次编号', 'laner_changfang']], how = 'left', left_on = 'fbatchno', right_on = '批次编号')

#字段8 
batchbase_value_8 = batchbase_value_7.merge(baoyu_chanfang_num, how = 'left', on = 'fbatchno')

#字段9 
batchbase_value_9 = batchbase_value_8.merge(abortion_df_total.loc[:, ['fbatchno', 'abortion_rate']], how = 'left', on = 'fbatchno')

#添加字段=当前时间
batchbase_value_9['当前时间'] = currenttime=time.strftime('%Y-%m-%d',time.localtime(time.time())) #当前时间-年月日

In [165]:
batchbase_value = batchbase_value_9.loc[:, ['当前时间','ffieldid', 'ffield', 'fareaid', 'farea', 'fregionname', 'fbigregionname', 'fbatchno', 'fsegmentid', 'source_fieldnum', 'laner_houbei', '大圈配种怀孕批次', 'laner_huaiyun', 
                                           'day_gap_60', '保育批次是否是大圈配种母猪分娩后代', 'laner_changfang', 'source_chanfang_num', 'abortion_rate']]

In [166]:
#找到污染源
pollution_batch = batchbase_value.dropna(subset = ['source_fieldnum', 'laner_houbei', '大圈配种怀孕批次', 'laner_huaiyun', 
                                           'day_gap_60', '保育批次是否是大圈配种母猪分娩后代', 'laner_changfang', 'source_chanfang_num', 'abortion_rate'], how = 'all').index 
#对污染源添加字段 pollution_batch
batchbase_value.loc[pollution_batch , '是否污染源'] = '是'

In [167]:
#划分等级, 1,2,3等级分别对应 高风险，中风险，低风险
batchbase_value_copy = batchbase_value.copy()

#该场区后备母猪来源涉及场区个数>1， 所有场区高风险
highrisk_field_1 = list(batchbase_value.loc[batchbase_value.source_fieldnum > 1].ffieldid.unique()) #高风险场区列表
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_1), ['risk_level1', 'reason1']] = 1, '该场区后备母猪来源涉及场区个数>1'

#该场区后备猪进群前检出蓝耳抗原， 所有场区高风险
highrisk_field_2 = list(batchbase_value.loc[batchbase_value.laner_houbei.notnull()].ffieldid.unique()) 
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_2), ['risk_level2', 'reason2']] = 1, '该场区后备猪进群前检出蓝耳抗原'

#一胎怀孕母猪是大圈配种的批次, 仅指这个批次
batchbase_value_copy.loc[batchbase_value.大圈配种怀孕批次.notnull(), ['risk_level3', 'reason3']] = 1, '该批次是一胎怀孕母猪是大圈配种的批次'

#该场区怀孕批次检出蓝耳抗原个数>1， 所有场区中风险
highrisk_field_4 = list(batchbase_value.loc[batchbase_value.laner_huaiyun.notnull()].ffieldid.unique())
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_4), ['risk_level4', 'reason4']] = 2, '该场区怀孕批次检出蓝耳抗原个数>1'

#该场区后备母猪隔离期<60， 所有场区中风险
highrisk_field_5 = list(batchbase_value.loc[batchbase_value.day_gap_60.notnull()].ffieldid.unique())
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_5), ['risk_level5', 'reason5']] = 2, '该场区后备母猪隔离期<60'

#该保育批次是大圈配种母猪分娩后代，仅指这个批次
batchbase_value_copy.loc[batchbase_value.保育批次是否是大圈配种母猪分娩后代.notnull(), ['risk_level6', 'reason6']] = 1, '该保育批次是大圈配种母猪分娩后代'

#该产房批次病原检出批次个数>1
batchbase_value_copy.loc[batchbase_value.laner_changfang > 1, ['risk_level7', 'reason7']] = 2, '该产房批次病原检出批次个数>1'

#保育批次接猪混群单元>=2(产房单元>=2)，仅指这个批次
batchbase_value_copy.loc[batchbase_value.source_chanfang_num >= 2, ['risk_level8', 'reason8']] = 2, '保育批次接猪混群单元>=2(产房单元>=2)'

#怀孕批次流产率>3% = 高风险, 1.5%<怀孕批次流产率≤3% = 中风险, 怀孕批次流产率≤1.5% = 低风险
batchbase_value_copy.loc[batchbase_value.abortion_rate > 0.03, ['risk_level9', 'reason9']] = 1, '怀孕批次流产率>3%'
batchbase_value_copy.loc[(batchbase_value.abortion_rate <= 0.03)&(batchbase_value.abortion_rate > 0.015), ['risk_level7', 'reason7']] = 2, '1.5%<怀孕批次流产率≤3%'
batchbase_value_copy.loc[batchbase_value.abortion_rate <= 0.015, ['risk_level9', 'reason9']] = 3, '怀孕批次流产率≤1.5%'

#该场区后备母猪来源涉及场区个数 = 1 低风险
highrisk_field_6 = list(batchbase_value.loc[batchbase_value.source_fieldnum == 1].ffieldid.unique()) #风险场区列表
batchbase_value_copy.loc[batchbase_value.ffieldid.isin(highrisk_field_6), ['risk_level10', 'reason10']] = 3, '该场区后备母猪来源涉及场区个数=1'

In [168]:
#risk_level所有数值权为空则为 ’无法判断“等级
batchbase_value_nonnull = batchbase_value_copy.dropna(subset = ['risk_level1','risk_level2', 'risk_level3', 'risk_level4', 'risk_level5', 'risk_level6', 'risk_level7', 'risk_level8', 'risk_level9', 'risk_level10'], how = 'all')

# 每一行第一个非空数字即为风险等级
risk_values = batchbase_value_nonnull.loc[:, ['risk_level1','risk_level2', 'risk_level3', 'risk_level4', 'risk_level5', 'risk_level6', 'risk_level7', 'risk_level8','risk_level9', 'risk_level10']].values 
batchbase_value_nonnull["risk_level_no"] = [risk_values[x][np.where(~np.isnan(risk_values[x]))][0] for x in range(len(batchbase_value_nonnull))]
risk_dict = {1:'高风险', 2:'中风险', 3:'低风险'} #风险等级字典
batchbase_value_nonnull['风险等级'] = batchbase_value_nonnull.risk_level_no.replace(risk_dict)
#填充到原来的基础表，index不变
batchbase_value_copy['风险等级'] = batchbase_value_nonnull['风险等级']
#其他没有划分等级的批次, 填充为”无法判断“
batchbase_value_copy.风险等级 = batchbase_value_copy.风险等级.fillna('无法判断')

#等级划分原因罗列
reason_list = ['reason1', 'reason2', 'reason3', 'reason4', 'reason5', 'reason6', 'reason7', 'reason8', 'reason8', 'reason9', 'reason10']
batchbase_value_copy['划分原因'] = batchbase_value_copy.loc[:,reason_list].apply(lambda x: ','.join(x.dropna()), axis = 1)


/tmp/ipykernel_15563/420839497.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batchbase_value_nonnull["risk_level_no"] = [risk_values[x][np.where(~np.isnan(risk_values[x]))][0] for x in range(len(batchbase_value_nonnull))]
/tmp/ipykernel_15563/420839497.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batchbase_value_nonnull['风险等级'] = batchbase_value_nonnull.risk_level_no.replace(risk_dict)


In [169]:
batchbase_value.loc[:, ['风险等级', '划分原因']] = batchbase_value_copy.loc[:, ['风险等级', '划分原因']]
# 工段
segment_dict = {13910:'泌乳母猪', 13909:'怀孕', 13905:'哺乳仔猪',13906:'保育', 13907:'育肥', 21254: '空怀', 29535:'待配', 13908:'后备'}

batchbase_value.fsegmentid = batchbase_value.fsegmentid.replace(segment_dict)
batchbase_value.是否污染源 = batchbase_value.是否污染源.fillna('否')


# 保存

In [170]:
#改成保存格式
save_batchvalue_df = batchbase_value.rename(columns = {'ffieldid':'所在场区ID', 'ffield':'所在场区名', 'fareaid':'所在子公司id','farea':'所在子公司名', 'fregionname': '区域', 'fbigregionname': '大区',
                                                       'fbatchno':'批次号', 'fsegmentid':'工段','source_fieldnum':'该场区母猪来源涉及场区个数', 'laner_houbei':'该场区后备猪批次进群前是否检出蓝耳抗原', 
                                                        'laner_huaiyun':'该场区怀孕批次检出蓝耳抗原个数', 
                                                       'day_gap_60':'该场区后备母猪批次驯化隔离期是否<60', 'laner_changfang':'该场区产房批次是否检出蓝耳病原','source_chanfang_num':'保育批次猪只产房来源个数', 'abortion_rate':'怀孕批次流产率'})
save_batchvalue_df

,当前时间,所在场区ID,所在场区名,所在子公司id,所在子公司名,区域,大区,批次号,工段,该场区母猪来源涉及场区个数,该场区后备猪批次进群前是否检出蓝耳抗原,大圈配种怀孕批次,该场区怀孕批次检出蓝耳抗原个数,该场区后备母猪批次驯化隔离期是否<60,保育批次是否是大圈配种母猪分娩后代,该场区产房批次是否检出蓝耳病原,保育批次猪只产房来源个数,怀孕批次流产率,是否污染源,风险等级,划分原因
0,2022-01-25,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,宛东,中原,SQ1327C00306049461220107010,泌乳母猪,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,低风险,该场区后备母猪来源涉及场区个数=1
1,2022-01-25,15375746,社旗牧原7场繁殖场,15168980,社旗牧原,宛东,中原,SQ1327C00205084430220106011,怀孕,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,中风险,"该场区怀孕批次检出蓝耳抗原个数>1,该场区后备母猪隔离期<60,该场区后备母猪来源涉及场区个数=1"
2,2022-01-25,15337422,馆陶牧原9场生长场,15314426,馆陶牧原,京津冀,中原,GT0433D00102133938220106084,保育,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,低风险,该场区后备母猪来源涉及场区个数=1
3,2022-01-25,15383121,社旗牧原4场繁殖场,15168980,社旗牧原,宛东,中原,SQ1327C00301049461220107010,哺乳仔猪,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,低风险,该场区后备母猪来源涉及场区个数=1
4,2022-01-25,15470599,宁晋牧原12场综合场,15388059,宁晋牧原,京津冀,中原,NJ0528C00501036130220106003,哺乳仔猪,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,低风险,该场区后备母猪来源涉及场区个数=1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,2022-01-25,15436747,乌拉特牧原2场生长场,15436743,乌拉特牧原,内蒙古,东北,WULQQ0823D00103081233220119081,育肥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,否,低风险,该场区后备母猪来源涉及场区个数=1
90881,2022-01-25,15432159,新绛牧原7场生长场,15281045,新绛牧原,山西,西北,XJ00003250315211110082,育肥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.102941,是,高风险,"怀孕批次流产率>3%,该场区后备母猪来源涉及场区个数=1"
90882,2022-01-25,15416804,夏县牧原8场生长场,15338653,夏县牧原,山西,西北,XX00002032269211222020,保育,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.141026,是,高风险,怀孕批次流产率>3%
90883,2022-01-25,15413631,颍上牧原5场生长场,15336875,颍上牧原,安徽,中原,YS00002031111211119009,保育,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.138620,是,高风险,怀孕批次流产率>3%


In [171]:
save_batchvalue_df.风险等级.value_counts()

高风险     69611
中风险     12329
低风险      5799
无法判断     3146
Name: 风险等级, dtype: int64

In [173]:
#save_batchvalue_df.to_csv('蓝耳批次风险划分-繁殖-生长-综合场.csv', index = False)